**4주차**

다변수 처리

In [ ]:
class Function:

    def __call__(self,*inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) #다 풀어서 전달(리스트 언팩)
        if not isinstance(ys,tuple): #자료형 검사
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
            self.inputs =inputs
            self.outputs = outputs
            return outputs if len(outputs) > 1 else outputs[0]
        
    def forward(self,x):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise not NotImplementedError()

In [ ]:
class Add(Function):
    def forward(self,x0,x1): #func의 call로 들어가므로 간단히 구현 가능
        y = x0 + x1
        return y #튜플
    
    def backward(self,gy):
        return gy, gy

In [ ]:
def add(x0,x1):
    return Add()(x0,x1)

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0,x1)
print(y.data)

z11 = w1x1 + w2x2 + .... 


인데 얘를 하나씩 미분하면  z111/dx1 = w1 .... (나머진 다 상수)

In [ ]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad= None
        self.creator = None

    #나의 크리에이터가 누구냐를 지정
    def set_creator(self,func):
        self.creator = func

    '''def backward(self): #13p 재귀적 호출
        f = self.creator
        if f is not None: #반복작업 자동화
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()'''

    def backward(self): #3.16p
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # f1,f2,f3,... 순차적으로 꺼냄
            """x,y = f.input, f.output
            x.grad = f.backward(y.grad) #y의 grad값을 집어넣으면 x.grad가 나옴"""
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys) #역전파해야 미분값이 나오니까
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
            for x, gx in zip(f.inputs,gxs): #zip 쓴 이유: 레이어별 미분값이 있을텐데 묶어둠
                x.grad = gx  # 같은 변수 사용되는 문제 해결
 
            if x.creator is not None:                   
                funcs.append(x.creator)

In [ ]:
#zip이란
list1 = [1,2,3]
list2 = ['a','b','c']
list3 = ['x','y','z']
zipped = zip(list1,list2,list3)
list(zipped)

시퀀스도 가변 길이 입출력 지원하도록

In [ ]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        #"""x = self.input.data"""
        x = self.input[0].data
        gx = 2 * x * gy
        return gx

In [ ]:
def square(x):
    return Square()(x)

def add(x0,x1):
    return Add()(x0,x1)

In [ ]:
x = Variable(np.array(2))
y = Variable(np.array(3))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad , y.grad)

1. 같은 변수를 반복했을 때의 문제
2. 덮어써버리는 문제

In [ ]:
def cleargrad(self):
    self.grad = None

In [ ]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()
print(x.grad)

x.cleargrad()
y = add(x,x)
y.backward()
print(x.grad)

p21

c를 처리한 이후 리스트에 B,A가 남는데
funcs.pop 하니까 A가 먼저 남고 B가 되니까 꼬여서 미분값이 달라진다.


-> 이 문제 해결을 위해 ->

함수 우선순위 설정

In [1]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 #세대를 나타내는 인스턴스변수

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1 #부모의 함수세대보다 1만큼 큰 값 설정

## 5주차<br> 
#### 딥프 기본 구현사항 패키지화

In [2]:
#메모리 관리와 순환을 위해 DeZero의 성능을 개선해야한다.
#이번 시간은 사용자 편의성을 위해. 유저 인터페이스를 개선하고 최종적으로 패키지화 시킬 것이다.
#step들을 가져다가 import해서 씀

파이썬의 메모리 관리 두 가지 방식

1. reference

In [3]:
class obj:
    pass
def f(x):
    print(x)
a = obj() # a객체 생성 후 reference count 1
f(a) # reference count 2
#함수 완료: 빠져나오면 1
a = None # 대입 해제: 카운트 0, 회수

function 입장에서 input과 output이 있고, Variable 입장에선 부모는 Function이다<br>
복잡한 곳인데 계속 순환 참조하면 메모리가 엄청 커져 학습시간이 늘어나기에 순환참조를 만들지않는 것이 좋다

In [4]:
# 약한 참조 
# 다른 객체를 참조하되 카운터는 증가시키지않는 기능

import weakref
import numpy as np
a = np.array([1,2,3])
b = weakref.ref(a)
b


<weakref at 0x00000140D073DC60; to 'numpy.ndarray' at 0x00000140D0750ED0>

In [5]:
b()

array([1, 2, 3])

In [6]:
a= None
b

<weakref at 0x00000140D073DC60; to 'numpy.ndarray' at 0x00000140D0750ED0>

step_17

In [ ]:
class Config: #설정 데이터는 단 한군데서만 존재하는 것이 좋음.
    enable_backprop = True   #역전파가 가능한지 여부를 뜻하는 속성
    
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop: #
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [ ]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        #"""x = self.input.data"""
        x = self.input[0].data
        gx = 2 * x * gy
        return gx

#### DeZero의 메모리 절약 모드

**메모리 사용 개선 두가지 구조 도입**

1. 역전파 시 사용하는 메모리양을 줄이는 방법<br>
2. 역전파가 필요 없는 경우용 모드를 제공

In [7]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad = False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            #gys = [output.grad for output in f.outputs]
            gys = [output().grad for output in f.outputs]  # weakref 참조로 변경
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
        if not retain_grad:
            for y in f.outputs:
                y().grad = None

In [ ]:
class Add(Function):
    def forward(self,x0,x1): #func의 call로 들어가므로 간단히 구현 가능
        y = x0 + x1
        return y #튜플
    
    def backward(self,gy):
        return gy, gy
    
def add(x0, x1):
    return Add()(x0, x1)

In [ ]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0,x1)
y = add(x0,t)
y.backward()

print(y.grad, t.grad) #None None
print(x0.grad, x1.grad)

# 왜 둘다 None이 아닌가

모드 전환 코드 수행

In [ ]:
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

In [ ]:
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)

using_config 함수 구현 

set은 삽입 get은 꺼내기

In [8]:
import contextlib

In [ ]:
@contextlib.contextmanager
def using_config(name, value):  #with 블록에 들어갈 때 name 으로 지정한 Config 클래스 속성이 value로 설정
    old_value = getattr(Config, name) 
    setattr(Config, name, value) 
    try:
        yield
    finally:
        setattr(Config, name, old_value) 

def no_grad(): #역전파에 적용,
    return using_config('enable_backprop', False)

In [ ]:
#지금까지는 메모리 효율을 위해 순환참조를 없애버렸다
# 두번째는 메모리 절약(미분값 없애고 모드로 조절하는 법 했다)


#그 다음 19장 하는 것은 변수 사용성 개선

#### 변수 사용성 개선

1. 변수 이름 지정

In [ ]:
# 변수들을 서로 구분 
class Variable: 
    def __init__(self, data, name=None): #변수에 이름을 붙여줄 수 있도록 설정함.
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name # x = Variable(np.array(1.0), ‘input_x’)라면, 변수 x의 이름은 input_x 임
        self.grad = None
        self.creator = None
        self.generation = 0

2. ndarray 인스턴스 변수 
- Variable  인스턴스를 ndarray 인스턴스처럼 보이게 함

In [ ]:
# @property가 있어야 
#원래는 x.shape()이지만 x.shape으로 호출 가능

3. len과 print를 Variable에 쓸 수 있도록

In [ ]:
#__len__을 추가함으로서

# __repr__(self) : 나의 형식에 맞게 재정의 가능함. 

#### 연산자 오버로드(1) 
- 연산자를 지원하도록 Variable 확장 (ndarray끼리 +,* 을 지원)

mul() 쓰려고 지정했다가 그것보다 더 편리하게 오버로드로 *을 사용가능하게끔 했다

#### 연산자 오버로드(2)

a * np.array(2.0), 3 + b 가능

#### float, int와 함께 사용하기

In [ ]:
#21 전체 코드
import weakref
import numpy as np
import contextlib

class Config:
    enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

def no_grad():
    return using_config('enable_backprop', False)

class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref

def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [ ]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

#### 연산자 오버로드 (3)  

### 여태까지 우리는 자동미분하는 방법을 설계했다 이제는 패키지로 정리